# Simple Multi-Agent Compliance Report ServiceThis notebook demonstrates a local, end-to-end multi-agent workflow for generating standardized compliance reports using Google ADK. Everything stays within the Kaggle environment with no external network calls beyond package installation.

## 1. SetupInstall required packages and import common utilities plus the Google ADK components used across the notebook.

In [ ]:
# Install dependencies (kept in the notebook so it works in fresh Kaggle runtimes)
!pip install -q google-adk PyPDF2

import json
from typing import Dict, Any, List
from datetime import datetime
from pathlib import Path

from google.adk.agents import Agent
from google.adk.tools.function_tool import FunctionTool
from google.adk.sessions import Session

MODEL_NAME = "gemini-2.5-flash"


## 2. Project OverviewGoal: **Demonstrate a simple, locally hosted multi-agent service that generates standardized compliance reports for either code files or PDF business decks, using Google ADK with multiple agents, tools, sessions, and long-term memory.**The five agents and their roles are:- **Reception Agent:** Collects company and file information, classifies the file, and routes processing.- **PDF Processor Agent:** Reads PDF-like files and extracts simple metrics.- **Code Processor Agent:** Reads code files and extracts simple metrics.- **Report Generator Agent:** Produces structured report drafts using analysis metrics and any evaluator feedback.- **Evaluating Agent:** Critiques draft reports and suggests improvements until they look mature.All tools accept JSON **strings** as input and return JSON-formatted **strings** to keep I/O consistent.

## 3. Long-term Memory & Helper Data StructuresWe maintain a lightweight long-term memory to demonstrate persistence across runs. Helper functions manage memory entries and a tiny supported companies list keeps examples grounded.

In [ ]:
# Simple long-term memory for demo purposes
LONG_TERM_MEMORY: Dict[str, List[Dict[str, Any]]] = {
    "reports": []
}

SUPPORTED_COMPANIES = ["DemoCorp", "AcmeAnalytics", "Globex"]


def add_report_to_memory(company: str, file_path: str, report: Dict[str, Any]) -> Dict[str, Any]:
    """Append a compact report summary to long-term memory."""
    LONG_TERM_MEMORY["reports"].append(
        {
            "company": company,
            "file_path": str(Path(file_path)),
            "timestamp": datetime.utcnow().isoformat(),
            "summary": report.get("summary", ""),
            "risk_score": report.get("risk_score", 0),
        }
    )
    return {
        "status": "saved",
        "total_reports": len(LONG_TERM_MEMORY["reports"]),
    }


def lookup_reports(company: str, file_path: str) -> List[Dict[str, Any]]:
    """Return prior reports for the same company or file if present."""
    normalized_path = str(Path(file_path))
    return [
        r
        for r in LONG_TERM_MEMORY["reports"]
        if r.get("company") == company or r.get("file_path") == normalized_path
    ]


## 4. Tool Implementations (string in -> JSON string out)Each tool immediately parses the JSON string payload and always returns JSON-encoded strings. They are wrapped with `FunctionTool` so Google ADK agents can call them.

In [ ]:
# Tool 1: File info and routing helper

def file_info_tool_fn(payload: str) -> str:
    """Classify a file path as code or pdf-like and normalize paths."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = Path(data.get("file_path", ""))
    suffix = file_path.suffix.lower()
    file_type = "pdf" if suffix == ".pdf" else "code"
    result = {
        "company": company,
        "file_path": str(file_path),
        "file_type": file_type,
    }
    return json.dumps(result)


# Tool 2: PDF reader

def pdf_reader_tool_fn(payload: str) -> str:
    """Read PDF contents; fall back to placeholder text if reading fails."""
    data = json.loads(payload)
    file_path = Path(data.get("file_path", ""))
    extracted_text = ""
    page_count = 0
    if file_path.exists() and file_path.suffix.lower() == ".pdf":
        try:
            from PyPDF2 import PdfReader

            reader = PdfReader(str(file_path))
            page_count = len(reader.pages)
            extracted_text = "
".join(page.extract_text() or "" for page in reader.pages)
        except Exception:
            extracted_text = "Example PDF content fallback."
    else:
        extracted_text = file_path.read_text() if file_path.exists() else "Example PDF content fallback."
        page_count = max(1, extracted_text.count("Slide"))
    result = {
        "file_path": str(file_path),
        "extracted_text": extracted_text or "Example PDF content fallback.",
        "page_count": page_count or 1,
    }
    return json.dumps(result)


# Tool 3: Code reader

def code_reader_tool_fn(payload: str) -> str:
    """Load code text with naive language detection."""
    data = json.loads(payload)
    file_path = Path(data.get("file_path", ""))
    language = "unknown"
    if file_path.suffix.lower() == ".py":
        language = "python"
    elif file_path.suffix.lower() == ".sql":
        language = "sql"
    code_text = file_path.read_text() if file_path.exists() else f"Placeholder snippet for {file_path.name}"
    result = {
        "file_path": str(file_path),
        "language": language,
        "code_text": code_text,
    }
    return json.dumps(result)


# Tool 4: PDF analysis

def pdf_analysis_tool_fn(payload: str) -> str:
    """Heuristically analyze PDF-like text for risk signals."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = data.get("file_path", "")
    extracted_text = data.get("extracted_text", "")
    lines = extracted_text.splitlines()
    estimated_slides = sum(1 for line in lines if "slide" in line.lower()) or max(1, len(lines))
    risky_terms = [term for term in ["guarantee", "confidential", "secret"] if term in extracted_text.lower()]
    contains_confidential = any(term in extracted_text.lower() for term in ["confidential", "secret"])
    result = {
        "file_path": file_path,
        "company": company,
        "metrics": {
            "estimated_slides": estimated_slides,
            "contains_confidential": contains_confidential,
            "risky_terms": risky_terms,
        },
    }
    return json.dumps(result)


# Tool 5: Code analysis

def code_analysis_tool_fn(payload: str) -> str:
    """Heuristically scan code for PII-like terms and logging usage."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = data.get("file_path", "")
    code_text = data.get("code_text", "")
    pii_terms = ["email", "ssn", "phone", "customer"]
    found_pii = [term for term in pii_terms if term.lower() in code_text.lower()]
    log_statement_count = code_text.count("print(") + code_text.count("logger.") + code_text.count("logging.")
    total_lines = len(code_text.splitlines()) if code_text else 0
    result = {
        "file_path": file_path,
        "company": company,
        "metrics": {
            "pii_mentions": found_pii,
            "log_statement_count": log_statement_count,
            "total_lines": total_lines,
        },
    }
    return json.dumps(result)


# Tool 6: Report drafting

def report_draft_tool_fn(payload: str) -> str:
    """Generate a structured compliance report draft from metrics."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = data.get("file_path", "")
    file_type = data.get("file_type", "unknown")
    analysis_metrics = data.get("analysis_metrics", {})
    previous_feedback = data.get("previous_feedback", "")

    risk_score = 20
    if file_type == "code":
        risk_score += 10 * len(analysis_metrics.get("pii_mentions", []))
        risk_score += 2 * analysis_metrics.get("log_statement_count", 0)
    else:
        risk_score += 15 * len(analysis_metrics.get("risky_terms", []))
        risk_score += 5 if analysis_metrics.get("contains_confidential") else 0
    risk_score = min(100, risk_score)

    summary = f"Draft compliance report for {company} covering {file_type} file {file_path}."
    if previous_feedback:
        summary += f" Incorporated feedback: {previous_feedback}."

    sections = {
        "overview": summary,
        "key_findings": [],
        "recommendations": [],
    }

    if file_type == "code":
        sections["key_findings"].append(
            f"PII mentions found: {', '.join(analysis_metrics.get('pii_mentions', [])) or 'none'}."
        )
        sections["key_findings"].append(
            f"Logging statements counted: {analysis_metrics.get('log_statement_count', 0)}."
        )
        sections["recommendations"].append("Review logging for sensitive data exposure.")
        if analysis_metrics.get("pii_mentions"):
            sections["recommendations"].append("Apply masking or tokenization to PII fields.")
    else:
        sections["key_findings"].append(
            f"Estimated slides: {analysis_metrics.get('estimated_slides', 'unknown')}."
        )
        sections["key_findings"].append(
            f"Confidential terms present: {analysis_metrics.get('contains_confidential', False)}."
        )
        sections["recommendations"].append("Ensure disclaimers are present for guarantees and promises.")
        if analysis_metrics.get("risky_terms"):
            sections["recommendations"].append("Highlight and qualify risky commitments in the deck.")

    report = {
        "company": company,
        "file_path": file_path,
        "file_type": file_type,
        "summary": summary,
        "risk_score": risk_score,
        "sections": sections,
    }
    return json.dumps(report)


# Tool 7: Report critique

def report_critique_tool_fn(payload: str) -> str:
    """Assess a draft report and suggest improvements."""
    data = json.loads(payload)
    report = data.get("report", {})
    recommendations = report.get("sections", {}).get("recommendations", [])
    risk_score = report.get("risk_score")
    missing_fields = []
    if not recommendations:
        missing_fields.append("recommendations")
    if risk_score is None:
        missing_fields.append("risk_score")

    is_satisfactory = len(missing_fields) == 0
    improvement_suggestions = []
    if not is_satisfactory:
        improvement_suggestions.append("Add actionable recommendations for the audience.")
    if risk_score is None:
        improvement_suggestions.append("Compute a risk_score between 0 and 100.")

    overall_comment = "Report looks solid." if is_satisfactory else "Please address the listed issues."
    result = {
        "is_satisfactory": is_satisfactory,
        "overall_comment": overall_comment,
        "improvement_suggestions": improvement_suggestions,
    }
    return json.dumps(result)


# Tool 8: Memory update

def memory_update_tool_fn(payload: str) -> str:
    """Persist a compact report summary to the in-notebook memory store."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = data.get("file_path", "")
    report = data.get("report", {})
    result = add_report_to_memory(company, file_path, report)
    return json.dumps(result)


# Tool 9: Memory lookup

def memory_lookup_tool_fn(payload: str) -> str:
    """Fetch previously saved reports for the same company or file path."""
    data = json.loads(payload)
    company = data.get("company", "UnknownCo")
    file_path = data.get("file_path", "")
    previous_reports = lookup_reports(company, file_path)
    return json.dumps({"previous_reports": previous_reports})


In [ ]:
# Wrap tool functions with FunctionTool for ADK agents
file_info_tool = FunctionTool.from_fn(
    file_info_tool_fn,
    name="file_info_tool",
    description="Classify files and normalize paths before routing.",
)
pdf_reader_tool = FunctionTool.from_fn(
    pdf_reader_tool_fn,
    name="pdf_reader_tool",
    description="Read PDF-like files and return extracted text.",
)
code_reader_tool = FunctionTool.from_fn(
    code_reader_tool_fn,
    name="code_reader_tool",
    description="Read code files with naive language detection.",
)
pdf_analysis_tool = FunctionTool.from_fn(
    pdf_analysis_tool_fn,
    name="pdf_analysis_tool",
    description="Heuristically analyze PDF text for risk signals.",
)
code_analysis_tool = FunctionTool.from_fn(
    code_analysis_tool_fn,
    name="code_analysis_tool",
    description="Heuristically analyze code for PII hints and logging.",
)
report_draft_tool = FunctionTool.from_fn(
    report_draft_tool_fn,
    name="report_draft_tool",
    description="Build a structured compliance report draft.",
)
report_critique_tool = FunctionTool.from_fn(
    report_critique_tool_fn,
    name="report_critique_tool",
    description="Critique a draft report and request fixes if needed.",
)
memory_update_tool = FunctionTool.from_fn(
    memory_update_tool_fn,
    name="memory_update_tool",
    description="Store reports in long-term memory.",
)
memory_lookup_tool = FunctionTool.from_fn(
    memory_lookup_tool_fn,
    name="memory_lookup_tool",
    description="Retrieve prior reports for a company or file.",
)


## 5. Agent Definitions (5 agents)Agents use the tools above to collaborate on compliance reports. Each agent keeps instructions concise and tool-focused so their outputs remain deterministic for the demo.

In [ ]:
# Reception agent to classify and route requests
reception_agent = Agent(
    name="reception_agent",
    model=MODEL_NAME,
    instructions=(
        "You are the reception and routing agent. Given a user message that includes "
        "a company name and a file path, you call tools to understand the file and "
        "then decide whether to send it to the PDF processor or the Code processor. "
        "Always use the tools and return a concise JSON-like routing summary."
    ),
    tools=[file_info_tool, memory_lookup_tool],
)

# PDF processor agent
pdf_processor_agent = Agent(
    name="pdf_processor_agent",
    model=MODEL_NAME,
    instructions=(
        "You process PDF business decks. You use tools to read the PDF and extract "
        "simple metrics that will be used later in a compliance report. You never make "
        "up metrics; you only summarize tool output."
    ),
    tools=[pdf_reader_tool, pdf_analysis_tool],
)

# Code processor agent
code_processor_agent = Agent(
    name="code_processor_agent",
    model=MODEL_NAME,
    instructions=(
        "You process analytics code files (Python, SQL, etc.). Use tools to read the "
        "code and compute simple metrics like PII mentions and logging counts for later "
        "reporting."
    ),
    tools=[code_reader_tool, code_analysis_tool],
)

# Report generator agent
report_generator_agent = Agent(
    name="report_generator_agent",
    model=MODEL_NAME,
    instructions=(
        "You generate a standardized compliance report in structured JSON. You call "
        "tools to transform analysis metrics and any previous feedback into a clearer, "
        "more complete report."
    ),
    tools=[report_draft_tool],
)

# Evaluating agent

evaluating_agent = Agent(
    name="evaluating_agent",
    model=MODEL_NAME,
    instructions=(
        "You evaluate draft compliance reports. You use tools to critique the report "
        "and decide if it is satisfactory. If not, you provide clear suggestions for "
        "the report generator to improve it. Stop once the report is satisfactory or "
        "after a small number of iterations."
    ),
    tools=[report_critique_tool, memory_update_tool],
)


## 6. Orchestration / Service FunctionThe helper below wires the agents together, demonstrates ADK sessions, and runs a simple critique loop between the report generator and evaluator.

In [ ]:
def run_compliance_service(
    user_message: str,
    company: str,
    file_path: str,
    max_iterations: int = 3,
) -> Dict[str, Any]:
    """End-to-end local compliance service orchestration."""
    session = Session()
    critique_history: List[Dict[str, Any]] = []

    # 1) Reception: classify the file
    reception_prompt = (
        f"User message: {user_message}. Company: {company}. File path: {file_path}. "
        "Use tools to classify the file and return JSON with company, file_path, and file_type."
    )
    reception_result = reception_agent.run(reception_prompt, session=session)
    routing_output = getattr(reception_result, "output", reception_result)
    try:
        routing_dict = json.loads(routing_output) if isinstance(routing_output, str) else routing_output
    except Exception:
        routing_payload = json.dumps({"company": company, "file_path": file_path})
        routing_dict = json.loads(file_info_tool_fn(routing_payload))

    file_type = routing_dict.get("file_type", "code")
    normalized_path = routing_dict.get("file_path", file_path)

    # 2) Dispatch to processor agents
    if file_type == "pdf":
        reader_payload = json.dumps({"file_path": normalized_path})
        pdf_text = json.loads(pdf_reader_tool_fn(reader_payload))
        analysis_payload = json.dumps(
            {
                "company": company,
                "file_path": normalized_path,
                "extracted_text": pdf_text.get("extracted_text", ""),
            }
        )
        analysis = json.loads(pdf_analysis_tool_fn(analysis_payload))
        metrics = analysis.get("metrics", {})
    else:
        reader_payload = json.dumps({"file_path": normalized_path})
        code_text = json.loads(code_reader_tool_fn(reader_payload))
        analysis_payload = json.dumps(
            {
                "company": company,
                "file_path": normalized_path,
                "code_text": code_text.get("code_text", ""),
            }
        )
        analysis = json.loads(code_analysis_tool_fn(analysis_payload))
        metrics = analysis.get("metrics", {})

    # 3) Critique loop between generator and evaluator
    previous_feedback = ""
    final_report: Dict[str, Any] = {}

    for iteration in range(1, max_iterations + 1):
        draft_payload = json.dumps(
            {
                "company": company,
                "file_path": normalized_path,
                "file_type": file_type,
                "analysis_metrics": metrics,
                "previous_feedback": previous_feedback,
            }
        )
        draft_report = json.loads(report_draft_tool_fn(draft_payload))

        critique_payload = json.dumps(
            {
                "report": draft_report,
                "company": company,
                "file_path": normalized_path,
            }
        )
        critique = json.loads(report_critique_tool_fn(critique_payload))
        critique["iteration"] = iteration
        critique_history.append(critique)

        if critique.get("is_satisfactory"):
            final_report = draft_report
            break

        previous_feedback = "; ".join(critique.get("improvement_suggestions", []))
        final_report = draft_report

    # 4) Save to long-term memory
    memory_payload = json.dumps(
        {"company": company, "file_path": normalized_path, "report": final_report}
    )
    memory_status = json.loads(memory_update_tool_fn(memory_payload))

    return {
        "final_report": final_report,
        "critique_history": critique_history,
        "memory_status": memory_status,
    }


## 7. Demo RunsCreate small sample artifacts and run the compliance service twice: once for code and once for a deck-like text file (treated as a PDF stand-in).

In [ ]:
# Create sample files
Path("sample_code.py").write_text(
    "import logging

email = 'test@example.com'
logging.info(f'user email: {email}')
"
)
Path("sample_deck.txt").write_text(
    "Slide 1: Welcome
Slide 2: We guarantee amazing returns.
Slide 3: Thank you.
"
)

# Run the compliance service for both examples
final_report_code = run_compliance_service(
    user_message="Please review this analytics script.",
    company="DemoCorp",
    file_path="sample_code.py",
)

final_report_deck = run_compliance_service(
    user_message="Please review this business deck.",
    company="DemoCorp",
    file_path="sample_deck.txt",
)


In [ ]:
import pprint

print("Final report for code:")
pprint.pp(final_report_code)
print("
Final report for deck-like file:")
pprint.pp(final_report_deck)


In [ ]:
from IPython.display import Markdown, display


def render_report_markdown(report_bundle: Dict[str, Any]) -> str:
    """Pretty render the final report dictionary into Markdown."""
    report = report_bundle.get("final_report", {}) if "final_report" in report_bundle else report_bundle
    sections = report.get("sections", {})
    key_findings = "
".join(f"- {item}" for item in sections.get("key_findings", []))
    recommendations = "
".join(f"- {item}" for item in sections.get("recommendations", []))
    md = f"""
### Compliance Report for {report.get('company')} ({report.get('file_type')})
*File:* {report.get('file_path')}  
*Risk score:* {report.get('risk_score')}

**Summary**  
{report.get('summary')}

**Key Findings**
{key_findings or '- None recorded.'}

**Recommendations**
{recommendations or '- None recorded.'}
"""
    return md

print("
Markdown view for code run:")
display(Markdown(render_report_markdown(final_report_code)))

print("
Markdown view for deck run:")
display(Markdown(render_report_markdown(final_report_deck)))


## 8. Simple Evaluation / ReflectionThe critique loop lets the evaluating agent request improvements until a report is satisfactory or the iteration limit is reached. Below we print how many iterations each demo consumed and show a tiny heuristic that checks whether higher-risk inputs produce higher scores.

In [ ]:
# Summarize critique iterations
code_iterations = len(final_report_code.get("critique_history", []))
deck_iterations = len(final_report_deck.get("critique_history", []))
print(f"Code report iterations: {code_iterations}")
print(f"Deck report iterations: {deck_iterations}")

# Simple heuristic evaluation across synthetic cases
synthetic_cases = [
    {
        "description": "Low-risk code",
        "metrics": {"pii_mentions": [], "log_statement_count": 1, "total_lines": 10},
        "file_type": "code",
    },
    {
        "description": "PII-heavy code",
        "metrics": {"pii_mentions": ["email", "customer"], "log_statement_count": 4, "total_lines": 50},
        "file_type": "code",
    },
    {
        "description": "Risky deck",
        "metrics": {"estimated_slides": 5, "contains_confidential": True, "risky_terms": ["guarantee"]},
        "file_type": "pdf",
    },
]

for case in synthetic_cases:
    payload = json.dumps(
        {
            "company": "EvalCorp",
            "file_path": f"/tmp/{case['description'].replace(' ', '_').lower()}",
            "file_type": case["file_type"],
            "analysis_metrics": case["metrics"],
            "previous_feedback": "",
        }
    )
    report = json.loads(report_draft_tool_fn(payload))
    print(f"{case['description']}: risk_score={report['risk_score']}")
